In [1]:
# imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import CoolProp.CoolProp as CP
import src.Fluid_CP as FCP
import src.Compressormodel_SP as CSP

**Assumptions**

- water basin and pasta can be modelled as 0D --> no temperature gradients inside water and pasta

- temperatures and pressure dependency of the properties of water and the pasta can be neglected

- subcritical heat pump process

- The pasta water is the heat sink

- Make sure the heat pump meets usual technical constraints (e.g., superheating) and is 
in COP-optimal operation at each point in time. 


**Steps**

1. Calculate temperature profile of water inside using controll strategy

2. Use this as an input to calculate heat demand 

3. Calculate condensation temperature 

4. Calculate the rest of the process through given limits (T_co, Tambient, Heat demand, various efficiencies)

5. reiterate through different fluids, conditions and sizes

In [10]:
# parameters

## basin geometry

V_water = (47-0.4) * (27-0.4) * (20-0.2) # cm^3

V_basket = 20 * 10 * 15 # cm^3 

rho_water=997. #kg/m3

## cooking pasta

#thermal storage

Tw_morning = 20 #degree Celsius

Tp_before = 7 #degree celsius

P_batch = 2 # kg --> 0.5 kg per basket

tc = 7 #minutes --> pasta is cooked for 7 minutes per batch

Tp_after = 85 #degree celsius

tb = 5 #minutes --> time between cooking batches

Tw_op_min = 85 #degree celsius --> minimal temperature during cooking

Tw_op_max = 93 #degree celsius --> maximal temperature during cooking

## other

R_env_w = 30 #K/kW --> evaporation of the cooking  water is neglected

R_p_w = 15 #K/kW 

c_pw = 4.18 #kj/kgK

c_pp = 3.5 #kj/kgK

## heatpump process

HP_log = [] # keeping track of on/off status

Tamb = 20 #degree celsius --> air temperature in cross flow

## compressor

D_comp = [35,40,45,50] #possible compressor sizes in mm

## Heat exchanger

d_outer = 6.35 #mm

d_inner = 5.55 #mm

## Heat transfer coefficients

h_r_li = 1000 #W/(m^2*K)

h_r_2ph = 2000 #W/(m^2*K)

h_r_va = 200 #W/(m^2*K)

h_w = 1000 #W/(m^2*K)


In [ ]:
def calc_HP(T,T_max,T_min):
    global HP, HP_log

    if HP==0 and T>T_max:
        Q_HP=0. #kW HP off
        HP=0
        HP_log.append(HP)
    if HP==0 and T<=T_min:
        Q_HP=5. #kW HP switched on --> call function to calculate heat 
        HP=1 #1=HP on
        HP_log.append(HP)
    if HP==1 and T<=T_min:
        Q_HP=5. #kW HP on --> call function to calculate heat 
        HP_log.append(HP)
    if HP==1 and T>T_max:
        Q_HP=0. #kW HP switched off
        HP=0
        HP_log.append(HP)
    return Q_HP

In [ ]:
def temp_water(T,t,Q_HP,m_store):

    Q_env=1./R*(T-T_env)/1000. #kW
    dTdt = (Q_HP-Q_out-Q_env)/(m_store*c_store)
    return dTdt

In [ ]:
from scipy.integrate import odeint

def T_water(T_min,T_max,V_water,n):
    
    #initial storage temperature
    T0 = T_max #fully charged
    # time points
    t = linspace(0,n,n)
    # store storage temperature
    T=zeros(n)
    # save initial storage temperature
    T[0] = T0
    # mass water in storage
    m_store=V_store/1e3*rho_water #kg
    
    # heat pump
    Q_HPs=zeros(n) #store power of heat pump
    Q_HPs[0]=0 #kW power of heat pump at t=0 is 0 kW

    # solve differential equation for the storage temperature
    for i in range(1,n):
        # span for next time step
        tspan = [t[i-1],t[i]]
        # solve for next step
        # needs to be modified to account for with pasta or without
        z = odeint(temp_water,T0,tspan,args=(Q_HPs[i-1],m_store))
        #heat pump power for netxt time step
        Q_HPs[i]=calc_HP(z[1],T_min,T_max)
        # store storage temperature
        T[i] = z[1]
        # next initial condition
        T0 = z[1]

    return T

In [15]:
# empirical calculation of resulting heatpump COP

def COP_res(COP_inner, Q_demand, Q_heatpump):
    return COP_inner * ((Q_demand / Q_heatpump) / 0.9*(Q_demand / Q_heatpump) + 0.1)



In [16]:
#refrigerant
delta_T_sh=10. #K #superheating
delta_T_sc=5. #K #subcooling
eta_is=0.52
fluid="R1234yf"
Eh="CBar"

#source
fluid_so="water"
T_so_in=10. #°C
T_so_out=5. #°C

#sink
fluid_si="water"
T_si_in=30. #°C
T_si_out=36. #°C

#heat exchanger
min_pinch=0.5 #K

In [9]:

def COP_inner(T_co,T_ev,delta_T_sh,fluid,Eh):
    #state 1*: evaporator: T1=T_ev and x1=1.0 (saturated vapor)
    state1_sat=FCP.state(["T","x"],[T_ev,1.0],fluid,Eh)
    #state 1: outlet evaporator: T1=T_ev+delta_T_sh and p1=p1_sat
    if delta_T_sh>0.:
        state1=FCP.state(["T","p"],[T_ev+delta_T_sh,state1_sat["p"]],fluid,Eh)
    else:
        state1=state1_sat
    
    #state 3: outlet condenser: T3=T_co and x3=0.0 (boiling liquid)
    state3=FCP.state(["T","x"],[T_co,0.0],fluid,Eh)
    
    #state 2s: isentropic compressor: p2s=p_co=p3 and s2s=s1 
    state2s=FCP.state(["p","s"],[state3["p"],state1["s"]],fluid,Eh)
    #enthalpy h2 is calulated using the compressor efficiency eta_comp$

    params = T_ev, T_co, delta_T_sh, 0, D_comp 

    eta_is, m_dot = CSP.recip_comp_SP(params,fluid)

    h2=state1["h"]+(state2s["h"]-state1["h"])/eta_is
    #state 2: outlet compressor: p2=p_co=p3 and h2
    state2=FCP.state(["p","h"],[state3["p"],h2],fluid,Eh)
    
    #state 4: outlet throttle: p4=p_ev=p1 and h4=h3 (isenthalpic throttle)
    state4=FCP.state(["p","h"],[state1["p"],state3["h"]],fluid,Eh)
    
    #spec. heat released in the condenser
    q_high=(state2["h"]-state3["h"]) #kJ/kg
    #spec. compressor work
    w_comp=state2["h"]-state1["h"]  #kJ/kg
    #coefficient of performance 
    COP=q_high/w_comp
    
    #thermodynamic mean temperature evaporator 
    T_m_v=(state1["h"]-state4["h"])/(state1["s"]-state4["s"])-273.15 #°C
    #thermodynamic mean temperature condenser
    T_m_c=(state3["h"]-state2["h"])/(state3["s"]-state2["s"])-273.15 #°C

    return [COP,state1,state2,state3,state4,T_m_v,T_m_c]

In [19]:
def pinch_con(param):
    T_co,T_ev=param
    #thermodynamic states of the heat pump process
    #state 1*: evaporator: T1=T_ev and x1=1.0 (saturated vapor)
    state1_sat=FCP.state(["T","x"],[T_ev,1.0],fluid,Eh)
    #state 1: outlet evaporator: T1=T_ev+delta_T_sh and p1=p1_sat
    if delta_T_sh>0.:
        state1=FCP.state(["T","p"],[T_ev+delta_T_sh,state1_sat["p"]],fluid,Eh)
    else:
        state1=state1_sat
    
    #state 3*: outlet condenser: T3=T_co and x3=0.0 (boiling liquid)
    state3_sat=FCP.state(["T","x"],[T_co,0.0],fluid,Eh)
    #state 3: outlet condenser: T3=T_co-delta_T_sc and p3=p3_stat
    state3=FCP.state(["T","p"],[T_co-delta_T_sc,state3_sat["p"]],fluid,Eh)
    
    #state 2s: isentropic compressor: p2s=p_co=p3 and s2s=s1 
    state2s=FCP.state(["p","s"],[state3["p"],state1["s"]],fluid,Eh)
    #enthalpy h2 is calulated using the compressor efficiency eta_is
    h2=state1["h"]+(state2s["h"]-state1["h"])/eta_is
    #state 2: outlet compressor: p2=p_co=p3 and h2
    state2=FCP.state(["p","h"],[state3["p"],h2],fluid,Eh)
    
    ##### Condenser outlet 
    #The temperature difference at the outlet of the condenser *delta_T_out_co* is the difference 
    #between the outlet temperature of the refrigerant *T_3* and the inlet temperature of the sink *T_si_in*.
    delta_T_out_co=state3["T"]-T_si_in

    ##### Condenser inlet
    #The temperature difference at the inlet of the condenser *delta_T_in_co* is the difference 
    #between the inlet temperature of the refrigerant *T_2* and the outlet temperature of the sink *T_si_out*.
    delta_T_in_co=state2["T"]-T_si_out
    
    ##### Dew line condenser (refrigerant)
    #The temperature difference at the inside of the condenser *delta_T_inside_co* is the difference between the condenser
    #temperature *T_con* and the temperature of the heat sink at the dew line of the refrigerant *T_si_inside*.
    
    #enthalpy of the refrigerant at the inlet/outlet of the condenser
    h_out_co=state3["h"]
    h_in_co=state2["h"]
    #enthalpy of saturated vapor at T_co and p_co: h_sat_Tco
    h_sat_Tco=FCP.state(["T","x"],[T_co,1.0],fluid,Eh)["h"]

    #Enthalpies of heat sink 
    h_si_in=FCP.state(["T","p"],[T_si_in,1.0],fluid_si,Eh)["h"]
    h_si_out=FCP.state(["T","p"],[T_si_out,1.0],fluid_si,Eh)["h"]

    #State of the heat sink at the point the refrigerant passes the dew line
    h_si_inside=h_si_out-(h_si_out-h_si_in)/(state2["h"]-state3["h"])*(state2["h"]-h_sat_Tco)
    T_si_inside=FCP.state(["p","h"],[1.0,h_si_inside],fluid_si,Eh)["T"]
    #temperature difference
    delta_T_inside_co=T_co-T_si_inside
    
    #return the values at inlet, outlet, and dew line as a vector
    return [delta_T_in_co,delta_T_out_co,delta_T_inside_co]


In [20]:
def pinch_evap(param):
    T_co,T_ev=param
    #thermodynamic states of the heat pump process
    #state 1*: evaporator: T1=T_ev and x1=1.0 (saturated vapor)
    state1_sat=FCP.state(["T","x"],[T_ev,1.0],fluid,Eh)
    #state 1: outlet evaporator: T1=T_ev+delta_T_sh and p1=p1_sat
    if delta_T_sh>0.:
        state1=FCP.state(["T","p"],[T_ev+delta_T_sh,state1_sat["p"]],fluid,Eh)
    else:
        state1=state1_sat
    
    #state 3*: outlet condenser: T3=T_co and x3=0.0 (boiling liquid)
    state3_sat=FCP.state(["T","x"],[T_co,0.0],fluid,Eh)
    #state 3: outlet condenser: T3=T_co-delta_T_sc and p3=p3_stat
    state3=FCP.state(["T","p"],[T_co-delta_T_sc,state3_sat["p"]],fluid,Eh)
    
    #state 2s: isentropic compressor: p2s=p_co=p3 and s2s=s1 
    state2s=FCP.state(["p","s"],[state3["p"],state1["s"]],fluid,Eh)
    #enthalpy h2 is calulated using the compressor efficiency eta_is
    h2=state1["h"]+(state2s["h"]-state1["h"])/eta_is
    #state 2: outlet compressor: p2=p_co=p3 and h2
    state2=FCP.state(["p","h"],[state3["p"],h2],fluid,Eh)
    #state 4: outlet throttle: p4=p_ev=p1 and h4=h3 (isenthalpic throttle)
    state4=FCP.state(["p","h"],[state1["p"],state3["h"]],fluid,Eh)
    
    ##### Evaporator outlet
    #The temperature difference at the outlet of the evaporator *delta_T_out_ev* is the difference 
    #between the inlet temperature of the source *T_so_in* and the outlet temperature of the refrigerant *T_1*.
    delta_T_out_ev=T_so_in-state1["T"]
    
    ##### Evaporator inlet
    #The temperature difference at the inlet of the evaporator *delta_T_in_ev* is the difference 
    #between the outlet temperature of the source *T_so_out* and the inlet temperature of the refrigerant *T_4*. 
    delta_T_in_ev=T_so_out-state4["T"]
    
    #return the values at inlet and outlet as a vector
    return [delta_T_in_ev,delta_T_out_ev]

In [21]:
#Syntax: bounds=[(T_co_min, T_co_max), (T_ev_min, T_ev_max)]
#Define appropriate bounds
bounds=[(T_si_in,50.),(-15.,T_so_in)] #(T_co_min, T_co_max), (T_ev_min, T_ev_max)

In [22]:
#For a function funct with three return values, nonlinear constaints are defined as follows:
#Syntax: NC1=NonlinearConstraint(func, [min_1,min_2,min_3],[max_1,max_2, max_3])
#Define the nonlinear constaints for the approach temperatures: NC1, NC2

NC2=NonlinearConstraint(pinch_con, [min_pinch,min_pinch,min_pinch],[np.inf,np.inf,np.inf])
#All constraints can be combined into a tuple
#Syntax:  NCs=(NC1, NC2,....)
NCs=(NC1,NC2)

NameError: name 'NonlinearConstraint' is not defined

In [23]:
#Define an appropriate initial guess (starting point).
T_co_start=40. #°C
T_ev_start=-5. #°C
#Use minimize to optimize the COP
Opti=minimize(calc_COP,x0=[T_co_start,T_ev_start],method='SLSQP',bounds=bounds,constraints=NCs)
#Check if the optimization terminated successfull and print the COP
print("Success optimization=",Opti.success)
COP=1./calc_COP(Opti.x)
print("optimized COP = ",round(COP,2))
#Print the optimized evaporator and condenser temperature: T_ev_op, T_co_op
T_ev_op=Opti.x[1]
T_co_op=Opti.x[0]
print("optimized condensation temperature= ",round(T_co_op,2),"°C")
print("optimized evaporation temperature= ",round(T_ev_op,2),"°C")

NameError: name 'minimize' is not defined